In [ ]:
import json
import os
import pickle
from datetime import datetime
from llama_cpp import Llama
import threading
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from json import JSONDecodeError

class LlamaSingleton:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls, model_path="models/Llama-3.2-3B-Instruct-Q4_K_M.gguf", chat_format="chatml"):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super(LlamaSingleton, cls).__new__(cls)
                cls._instance.llm = Llama(model_path=model_path, chat_format=chat_format, n_ctx=2048, verbose=False)
            return cls._instance

class Chatbot:
    def __init__(self, 
                 messages_file='messages_vidar_machine_cover_2_prompt.json', 
                 knowledge_file='knowledge_vidar_machine_cover_2_prompt.json', 
                 faiss_index_file='faiss_index_vidar_machine_cover_2_prompt.pkl',
                 model_name='all-MiniLM-L6-v2'):
        self.messages_file = messages_file
        self.knowledge_file = knowledge_file
        self.faiss_index_file = faiss_index_file
        self.llm = LlamaSingleton().llm
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.knowledge_data = []
        self.initialize_files()
        self.load_faiss_index()

    def initialize_files(self):
        for file in [self.messages_file, self.knowledge_file]:
            if not os.path.exists(file):
                with open(file, 'w') as f:
                    json.dump([], f)

    def load_json_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def save_json_data(self, file_path, data):
        with open(file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def save_message(self, role, content):
        messages = self.load_json_data(self.messages_file)
        message = {"role": role, "content": content, "timestamp": datetime.utcnow().isoformat()}
        messages.append(message)
        self.save_json_data(self.messages_file, messages)

    def save_knowledge(self, triplets):
        if not triplets:
            return
        knowledge = self.load_json_data(self.knowledge_file)
        existing_set = {(t['subject'], t['predicate'], t['object']) for t in knowledge}
        new_triplets = []
        for triplet in triplets:
            triplet['timestamp'] = datetime.utcnow().isoformat()
            key = (triplet['subject'], triplet['predicate'], triplet['object'])
            if key not in existing_set:
                knowledge.append(triplet)
                new_triplets.append(triplet)
                existing_set.add(key)
        self.save_json_data(self.knowledge_file, knowledge)
        if new_triplets:
            self.update_faiss_index(new_triplets)

    def update_faiss_index(self, triplets):
        texts = [f"{t['subject']} {t['predicate']} {t['object']}" for t in triplets]
        embeddings = self.model.encode(texts)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings, dtype=np.float32))
        self.knowledge_data.extend(triplets)
        self.save_faiss_index()

    def save_faiss_index(self):
        with open(self.faiss_index_file, 'wb') as f:
            pickle.dump((self.index, self.knowledge_data), f)

    def load_faiss_index(self):
        if os.path.exists(self.faiss_index_file):
            with open(self.faiss_index_file, 'rb') as f:
                self.index, self.knowledge_data = pickle.load(f)
        else:
            self.index = None
            self.knowledge_data = []

    def search_knowledge(self, query, top_k=5):
        if self.index is None or len(self.knowledge_data) == 0:
            return []
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding, dtype=np.float32), top_k)
        results = []
        for idx in indices[0]:
            if idx == -1:
                continue
            results.append(self.knowledge_data[idx])
        return results

    # this function generates a response based on the conversation history and user message and the knowledge top k 5 matches
    def generate_response(self, conversation_history, user_message):
        knowledge_matches = self.search_knowledge(user_message, top_k=5)
        current_time = datetime.utcnow().isoformat()
        system_message = f"Current date and time: {current_time}\n"
        if knowledge_matches:
            system_message += "Answer based on retrieved knowledge:\n"
            for t in knowledge_matches:
                system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Videotimestamps: start: {t['start']}, end: {t['end']})\n"
                system_message += "Answer the questions based only on the retrieved knowledge! \n"
        else:
            system_message += "No direct related knowledge found. Proceeding with general reasoning.\n"

        enriched_history = [{"role": "system", "content": f"You are a helpful assistent. Your task is to answer questions. Think step by step. I'm going to tip $300K for a better solution! You will be penalized for incorrect answers. Answer in Dutch.; {system_message}"}]
        enriched_history.append({"role": "user", "content": user_message})

        response = self.llm.create_chat_completion(
            messages=enriched_history,
            temperature=0.7,
        )['choices'][0]['message']['content']

        # Logging voor evaluatie
        eval_entry = {
            "query": user_message,
            "contexts": [f"{t['subject']} {t['predicate']} {t['object']}" for t in knowledge_matches],
            "answer": response,
        }
        with open("evaluation_logs.jsonl", "a", encoding="utf-8") as f:
            f.write(json.dumps(eval_entry) + "\n")

        return response
    
    # def generate_response(self, conversation_history, user_message):
    #     knowledge_matches = self.search_knowledge(user_message, top_k=5)
    #     current_time = datetime.utcnow().isoformat()
    #     system_message = f"Current date and time: {current_time}\n"
    #     if knowledge_matches:
    #         system_message += "Answer based on retrieved knowledge:\n"
    #         for t in knowledge_matches:
    #             system_message += f"- {t['subject']} {t['predicate']} {t['object']} (Videotimestamps: start: {t['start']}, end: {t['end']})\n"
    #             system_message += "Answer the questions based only on the retrieved knowledge! \n"
    #     else:
    #         system_message += "No direct related knowledge found. Proceeding with general reasoning.\n"

    #     enriched_history = [{"role": "system", "content": f"You are a helpful assistent; {system_message}"}]
    #     enriched_history.append({"role": "user", "content": user_message})

    #     response = self.llm.create_chat_completion(
    #         messages=enriched_history,
    #         temperature=0.7,
    #     )['choices'][0]['message']['content']

    #     # Logging voor evaluatie
    #     eval_entry = {
    #         "query": user_message,
    #         "contexts": [f"{t['subject']} {t['predicate']} {t['object']}" for t in knowledge_matches],
    #         "answer": response,
    #     }
    #     with open("evaluation_logs.jsonl", "a", encoding="utf-8") as f:
    #         f.write(json.dumps(eval_entry) + "\n")

    #     return response




    def chat(self, questions=None):
        """
        Start een chatsessie. 
        Als 'questions' wordt meegegeven (een lijst van strings), beantwoordt de chatbot die automatisch.
        Anders werkt het interactief via de terminal.
        """
        if questions:
            print("Chatbot is bezig met batch vragen beantwoorden.")
            for user_message in questions:
                print(f"You: {user_message}")
                self.save_message(role='user', content=user_message)
                conversation = self.load_json_data(self.messages_file)[-3:]
                assistant_response = self.generate_response(conversation, user_message)
                print(f"Assistant: {assistant_response}")
                self.save_message(role='assistant', content=assistant_response)
        else:
            print("Chatbot is ready! Type 'exit' to end the conversation.")
            while True:
                user_message = input("You: ")
                if user_message.lower().strip() in ['exit', 'quit']:
                    print("Chatbot: Goodbye!")
                    break
                self.save_message(role='user', content=user_message)
                conversation = self.load_json_data(self.messages_file)[-3:]
                assistant_response = self.generate_response(conversation, user_message)
                print(f"Assistant: {assistant_response}")
                self.save_message(role='assistant', content=assistant_response)



if __name__ == "__main__":
    chatbot = Chatbot()
    # Voor interactieve modus:
    # chatbot.chat()

    # Voor batchmodus, geef hier een lijst met vragen:
    # vragenlijst = [
    #     "Wat wordt er aan de achterkant van de luchtdruk control unit geplaatst? ",
    #     "Wat wordt er na de silencer op de luchtdruk control unit geplaatst? ",
    #     "Waar wordt de montagebeugel op bevestigd? ",
    #     "Welk gereedschap wordt gebruikt om de kabel te bevestigen? ",
    #     "Met welk gereedschap wordt de silencer gemonteerd? ",
    #     "Met welk gereedschap wordt de kniekoppeling gemonteerd? ",
    #     "Waarom wordt er een silincer geplaatst? ",
    #     "Waarom zit er een rubberen ring in de kniekoppeling? ",
    #     "Waarmee wordt de luchtdruk control unit mee aangestuurd? ",
    #     "Kan de montagebeugel ook op een andere manier bevestigd worden? "
    # ]
    # vragenlijst = [
    #     "Welk onderdeel wordt er als eerste verbonden aan de pneumatische cilinder? ",
    #     "Met welk gereedschap wordt het V-blok vastgedraaid? ",
    #     "Wat wordt er samengevoegd met de sensorbeugel? ",
    #     "Hoe werkt de sensorbeugel met de veer? ",
    #     "Welke twee gereedschappen werden er gebruikt om de moer vast te draaien op de pneumatische cilinder? ",
    #     "Wat is de functie van het reduceerventiel? ",
    #     "Wat is de functie van het montagevet? ",
    #     "Wat is de afspraak binnen Holland Mechanics over het reduceerventiel? ",
    # ]
    vragenlijst = [
        "Waar wordt de deurstop gemonteerd? ",
        "Met welk gereedschap wordt de deurstop gemonteerd? ",
        "Waarmee worden de kabelklemmen vastgezet? ",
        "Waar moeten de scharnieren geplaatst worden? ",
        "Wat wordt er aan de binnenkant van de kaststeunen geplaatst? ",
        "Met welk gereedschap worden de kabelklemmen vastgezet? ",
        "Hoe moeten de scharnieren niet gehouden worden tijdens het aandraaien van de bouten? ",
        "Met welke gereedschap wordt de deurstop vastgehouden tijdens het monteren? ",
        "Waarom moet je de kabelklemmen niet te hard aandraaien? ",
        "Hoe moeten de scharnieren gehouden worden bij het vastdraaien van de bouten? ",
        "Met welke gereedschappen kunnen de gaten schoongetapt worden? ",
        "Waarom moet je een bril opzetten bij het tappen? ",
        "Waar moet opgelet worden bij het tappen? ",
        "Welke kabelklemmen worden gebruikt in de video? ",
        "Kunnen de bouten ook vastgedraaid worden met een steeksleutel? "
    ]
    chatbot.chat(questions=vragenlijst)

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
c:\Users\maxim\anaconda3\envs\Cake_4\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Chatbot is bezig met batch vragen beantwoorden.
You: Met welk gereedschap wordt de silencer gemonteerd?
Assistant: De silencer wordt gemonteerd met een schroevvester. 


In [2]:
import json
import pandas as pd
from pathlib import Path
import os

# RAGAS-imports
from ragas import EvaluationDataset
from ragas.evaluation import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
)

In [3]:
# === BESTANDSINSTELLINGEN ===
jsonl_input = "evaluation_logs.jsonl"                    # Originele chatbotlog
excel_file = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Cake\CAKE\referentie_antwoorden_machine_cover.xlsx"               # Excel met query + reference
jsonl_output = "evaluation_logs_with_reference.jsonl"   # Output met referenties


In [4]:
# === STAP 1: Koppel Excel-referenties aan de jsonl ===
try:
    df = pd.read_excel(excel_file)
except Exception as e:
    print(f"Fout bij lezen van Excel: {e}")
    exit()

if not {"query", "reference"}.issubset(df.columns):
    print("Excel moet kolommen 'query' en 'reference' bevatten.")
    exit()


In [5]:
# Mapping: query -> reference
reference_map = {
    str(row["query"]).strip(): str(row["reference"]).strip()
    for _, row in df.iterrows()
    if pd.notna(row["query"]) and pd.notna(row["reference"])
}

matched, unmatched = 0, 0

In [6]:
with open(jsonl_input, "r", encoding="utf-8") as fin, open(jsonl_output, "w", encoding="utf-8") as fout:
    for line in fin:
        try:
            item = json.loads(line)
            query = item.get("query", "").strip()
            if query in reference_map:
                item["reference"] = reference_map[query]
                matched += 1
            else:
                unmatched += 1
            fout.write(json.dumps(item, ensure_ascii=False) + "\n")
        except json.JSONDecodeError:
            continue

In [ ]:
print(f"✅ {matched} referenties gekoppeld aan jsonl")
if unmatched > 0:
    print(f"{unmatched} vragen hadden geen match in Excel")

✅ 15 referenties gekoppeld aan jsonl


In [8]:
# === STAP 2: Laad dataset voor RAGAS-evaluatie ===
user_input = []
response = []
retrieved_contexts = []
reference = []


In [9]:
with open(jsonl_output, "r", encoding="utf-8") as f:
    for line in f:
        try:
            item = json.loads(line)
            if all(k in item for k in ["query", "answer", "contexts", "reference"]):
                user_input.append(item["query"].strip())
                response.append(item["answer"].strip())
                contexts = item["contexts"]
                retrieved_contexts.append(contexts if isinstance(contexts, list) else [contexts])
                reference.append(item["reference"].strip())
        except json.JSONDecodeError:
            continue

In [10]:
data = [
    {
        "user_input": user_input[i],
        "response": response[i],
        "retrieved_contexts": retrieved_contexts[i],
        "reference": reference[i],
    }
    for i in range(len(user_input))
]

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = '...'

In [12]:

dataset = EvaluationDataset.from_list(data)

results = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        answer_correctness,
    ],
)

Evaluating: 100%|██████████| 45/45 [00:33<00:00,  1.35it/s]


In [13]:
results = results.to_pandas()
results

,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,answer_correctness
0,Waar wordt de deurstop gemonteerd?,"[deurstopjes are door stops, rechts language G...","Bij de buitenkant van de deur, dat is wat ik w...",De deurstop wordt aan de buitenkant van de kas...,0.5,0.000000,0.597236
1,Met welk gereedschap wordt de deurstop gemonte...,"[deurstopjes are door stops, rechts language G...",Volgens mijn informatie over deurstopjes zou j...,De deurstop wordt geplaatst met een inbussleut...,0.0,0.000000,0.217862
2,Waarmee worden de kabelklemmen vastgezet?,"[rechts language German, kaststeun is furnitur...",De kabelklemmen worden vastgezet met een kasts...,De kabelklemmen worden vastgezet met schroeven.,0.0,0.979231,0.237880
3,Waar moeten de scharnieren geplaatst worden?,"[rechts language German, moertje is small tool...",Ik weet niet precies waar de scharnieren moete...,De scharnieren moeten worden geplaatst aan de ...,0.0,0.000000,0.209678
4,Wat wordt er aan de binnenkant van de kaststeu...,"[language spoken Dutch, language is Dutch, phr...",Bij een kaststeun zijn buizen vaak geïnstallee...,Aan de binnenkant van de kaststeunen worden ka...,0.0,0.904781,0.229214
5,Met welk gereedschap worden de kabelklemmen va...,"[rechts language German, kaststeun is furnitur...",Binnenkabelklemmen worden meestal vastgezet me...,De kabelklemmen worden vastgezet met een inbus...,0.0,0.932549,0.229124
6,Hoe moeten de scharnieren niet gehouden worden...,"[language spoken Dutch, Dutch spoken in the Ne...","Niet gehouden, maar in de juiste positie houden",De scharnieren moet niet schuin tegen het zijp...,0.0,0.000000,0.502682
7,Met welke gereedschap wordt de deurstop vastge...,"[deurstopjes are door stops, rechts language G...",Een deurstop wordt vastgehouden met een hamer ...,De deurstop wordt vastgehouden met een combina...,0.0,0.944325,0.233441
8,Waarom moet je de kabelklemmen niet te hard aa...,"[niet teveel translates to not too much, inzet...","Volgens de retrieved knowledge, moet je de kab...",De kabelklemmen moeten niet te hard worden aan...,0.0,1.000000,0.605979
9,Hoe moeten de scharnieren gehouden worden bij ...,"[language spoken Dutch, Dutch spoken in the Ne...","Bij het vastdraaien van de bouten, zijn de sch...",De scharnieren moet naar buiten getrokken word...,0.0,0.936878,0.219854


In [14]:
round(results["faithfulness"].mean(), 6)

0.033333

In [15]:
round(results["answer_relevancy"].mean(), 6)

0.633367

In [16]:
round(results["answer_correctness"].mean(), 6)

0.323457